In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import random

In [ ]:
def scale_data(data):
    num_features = data.shape[1]
    scale_params = np.zeros((num_features,2))
    mins = np.min(data, axis=0);maxes=np.max(data, axis=0)
    data = (data - mins) / (maxes - mins)
    # for i in range(num_features):
    #     min_val = np.min(data[:,i])
    #     max_val = np.max(data[:,i])
    #     scale_params[i,:] = [min_val, max_val]
    #     data[:,i] = (data[:,i] - min_val) / (max_val - min_val)
    return data, scale_params

data_df = pd.read_csv("data/final_dataset.csv")

data_df = data_df.drop(['Date'], axis=1)
data_df = data_df.iloc[-200:,:]
print(data_df.head())
data = np.nan_to_num(np.array(data_df, dtype=np.float32))
print(np.isnan(data).any().item())
data, scale_params = scale_data(data)
print(np.isnan(data).any().item())
# data, scale_params = scale_data(np.nan_to_num(np.array(data_df, dtype=np.float32)))
# print(data.shape)

In [ ]:
def create_sequence(data,seq_len):
    xs = []
    ys = []
    for i in range(len(data)-seq_len-1):
        x = data[i:(i+seq_len),:]
        # print(x.shape)
        y = data[i+seq_len,0]
        xs.append(x)
        ys.append(y)
    return np.array(xs),np.array(ys)

SEQ_LEN = 5
inputs , targets = create_sequence(data,SEQ_LEN)
inputs=torch.from_numpy(inputs);targets=torch.from_numpy(targets)
print(inputs.shape)
print(targets.shape)

In [ ]:

class PricePredictor(nn.Module):
    def __init__(self, input_size=12, hidden_layer_size=30, time_segment=5, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.time_segment_length = time_segment
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        # self.cell_double = (torch.zeros(1,self.hidden_layer_size), torch.zeros(1,self.hidden_layer_size))

    def forward(self, input_seq):
        _, cell_double = self.lstm(input_seq)
        # print(cell_double)
        predictions = self.linear(cell_double[0])
        # print(predictions.shape)
        return predictions


In [ ]:
BATCH_SIZE = 1
dataset = TensorDataset(inputs,targets)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset) - int(0.8*len(dataset))])

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

model = PricePredictor()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 194 
for i in range(epochs):
    # model.cell_double = (torch.zeros(1,model.hidden_layer_size), torch.zeros(1,model.hidden_layer_size))
    for seq, labels in train_loader:
        if not torch.isnan(seq).any().item():
            print("Voila")
        y_pred = model(seq)
        # print(y_pred)
        single_loss = loss_function(y_pred, labels)
        # optimizer.zero_grad()
        single_loss.backward()
        # optimizer.step()
    if random.random() < 0.1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    # if(i%5==0):

    #     print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')